In [1]:
import requests
import json, re, time
from datetime import datetime
import pandas as pd

In [2]:
with open('api.key','r') as f:
    APIKEY = f.read()

HEADERS = {'apikey': APIKEY,
            'accept': 'application/json',
            'content-type': 'application/json',
            'cookie': 'ASP.NET_SessionId=aiggen1ccck0gq141dgq1sip; ASP.NET_SessionId=aiggen1ccck0gq141dgq1sip'
          }

API_ENDPOINT = 'https://kong.speedcheckerapi.com:8443/ProbeAPIv2/'

probeInfoProperties = [
        "ASN",
        "CityName",
        "ConnectionType",
        "CountryCode",
        "DNSResolver",
        "GeolocationAccuracy",
        "IPAddress",
        "Latitude",
        "Longitude",
        "Network",
        "NetworkID",
        "Platform",
        "ProbeID",
        "Version"
    ]

In [3]:
def getTestSettings(cc, url):
    
    destinations = [url]
    json_test = {
                  "testSettings": {
                    "TestCount": 10,
                    "Sources": [
                      {
                        "CountryCode": cc,
                        "Platform": "Android"
                      }
                    ],
                    "Destinations": destinations,
                    "ProbeInfoProperties": probeInfoProperties
                  }
                }
    return json_test

In [4]:
def getTestSettingsEUUS(url):
    
    destinations = [url]
    json_test = {
                  "testSettings": {
                    "TestCount": 2,
                    "Sources": [
                      {
                        "CountryCode": "US"
                      },
                      {
                        "CountryCode": "FR"
                      },
                      {
                        "CountryCode": "DE"
                      },
                      {
                        "CountryCode": "NL"
                      },
                      {
                        "CountryCode": "GB"
                      }  
                    ],
                    "Destinations": destinations,
                    "ProbeInfoProperties": probeInfoProperties
                  }
                }
    return json_test

### StartPageLoadTest function

In [5]:
def startPageLoadTest(test_settings):
    
    test_url = API_ENDPOINT + "StartPageLoadTest"
    try:
        r = requests.post(test_url, data=json.dumps(test_settings), headers=HEADERS)
    except requests.exceptions.RequestException as e:
        return "Request FAILED"
    
    res = json.loads(r.text)
        
    if ("OK" == res['StartPageLoadTestResult']['Status']['StatusText']):
        return res['StartPageLoadTestResult']['TestID']
    else:
        return "FAILED"

### retrievePageLoadTestResults function

In [6]:
def retrievePageLoadTestResults(testID):
    
    url = API_ENDPOINT + "GetPageLoadResults?apikey=" + APIKEY + "&testID=" + testID
    
    try:    
        r = requests.get(url, headers=HEADERS)
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        return []
    
    res = json.loads(r.text)
    return res['PageLoadTestResults']

## Retrieve results

In [7]:
def getResultsFromTestIDs(df_tests_url):
    df_results = pd.DataFrame(columns=['domain',
                                       'cc',
                                       'ProbeInfo',
                                       'TestDateTime',
                                       'StatusCode',
                                       'StatusText',
                                       'DNSLookupTime', 
                                       'Destination', 
                                       'HTTPStatus', 
                                       'InitialConnection', 
                                       'NumberOfRequests', 
                                       'PageLoadTime', 
                                       'SSLNegotiationTime', 
                                       'TTFB', 
                                       'TotalDownloadedBytes'
                                      ])
    
    #Exclude FAILED items
    for index, row in df_tests_url.loc[df_tests_url.testID!='FAILED'].iterrows():
        
        domain = row['domain']
        cc = row['cc']
        testID = row['testID']
        pageLoadTestResults =  retrievePageLoadTestResults(testID)

        if (pageLoadTestResults is None or len(pageLoadTestResults) < 1):
            continue
        
        #print(pageLoadTestResults)
        #print("€€€€€€€€€€€€€€€€€€€€€€€€€€€€€€€€€€€")
        
        statusCode = ""
        statusText = ""
        dnsLookupTime = ""
        destination = ""
        hTTPStatus = ""
        initialConnection = ""
        numberOfRequests = ""
        pageLoadTime = ""
        sslNegotiationTime = ""
        ttfb = ""
        totalDownloadedBytes = ""
        
        for res in pageLoadTestResults:
        
            probeInfo = res['ProbeInfo']
            testDateTime = res['TestDateTime']

            #extract the epoch
            m = re.search('/Date\((\d+)\+0000\)/', testDateTime)
            if m:
                testDateTime = m.group(1)[:-3]

            #convert into human readable format
            testDateTime = datetime.fromtimestamp(int(testDateTime)).strftime('%Y-%m-%d')
            try:
                statusCode = res['TestStatus']['StatusCode']
                statusText = res['TestStatus']['StatusText']
                dnsLookupTime = res['DNSLookupTime']
                destination =  res['Destination']
                hTTPStatus = res['HTTPStatus']
                initialConnection = res['InitialConnection']
                numberOfRequests =  res['NumberOfRequests'] 
                pageLoadTime =  res['PageLoadTime'] 
                sslNegotiationTime = res['SSLNegotiationTime'] 
                ttfb = res['TTFB']
                totalDownloadedBytes = res['TotalDownloadedBytes']
            except:
                pass

            df_results = df_results.append({
                'domain': domain,
                'cc': cc,
                'ProbeInfo':probeInfo,
                'TestDateTime': testDateTime,
                'StatusCode':statusCode,
                'StatusText':statusText,
                'DNSLookupTime':dnsLookupTime,
                'Destination':destination,
                'HTTPStatus':hTTPStatus,
                'InitialConnection':initialConnection,
                'NumberOfRequests':numberOfRequests,
                'PageLoadTime':pageLoadTime,
                'SSLNegotiationTime':sslNegotiationTime,
                'TTFB':ttfb,
                'TotalDownloadedBytes':totalDownloadedBytes
            }, ignore_index=True)

    return df_results

In [8]:
def runMeasurements(df_links, url_type, filename):
    df_testID = pd.DataFrame(columns=['domain', 'cc', 'testID'])

    for index, row in df_links.iterrows():
        domain = row['domain']
        cc = row['cc']
        url = row[url_type]

        test_settings = getTestSettings(cc, url)
        testID = startPageLoadTest(test_settings)
        print(testID)
        
        df_testID = df_testID.append({'domain': domain, 'cc': cc, 'testID': testID}, ignore_index=True)
    
    df_testID.to_csv(filename, sep='|', encoding='utf-8', index=False)    
    

In [9]:
def runMeasurementsEUUS(df_links, url_type, filename):
    df_testID = pd.DataFrame(columns=['domain', 'cc', 'testID'])

    for index, row in df_links.iterrows():
        domain = row['domain']
        cc = row['cc']
        url = row[url_type]

        test_settings = getTestSettingsEUUS(url)
        testID = startPageLoadTest(test_settings)
        print(testID)
        
        df_testID = df_testID.append({'domain': domain, 'cc': cc, 'testID': testID}, ignore_index=True)
    
    df_testID.to_csv(filename, sep='|', encoding='utf-8', index=False)    

In [16]:
df_links = pd.read_csv('data/links_sample.csv', sep='|', encoding='utf-8', keep_default_na=False)

In [21]:
#runMeasurements(df_links, 'url', 'data/url_testID.csv')
#runMeasurements(df_links, 'ori_amp_url', 'data/ori_amp_url_testID.csv')
#runMeasurements(df_links, 'amp_viewer_url', 'data/amp_viewer_url_testID-21092018.csv')
#runMeasurements(df_links, 'amp_cdn_url', 'data/amp_cdn_url_testID_21092018.csv')
#runMeasurementsEUUS(df_links, 'amp_viewer_url', 'data/amp_viewer_url_testID_EUUS-21072018.csv')
runMeasurementsEUUS(df_links, 'amp_cdn_url', 'data/amp_cdn_url_testID_EUUS-22092018.csv')



523f5ac4-13a3-4517-a540-2d52e867fcdf
5d8694f2-8509-4f6a-bb63-b2908c63360e
f2693547-ffa8-47ac-ac5b-cbf3ff2fa3d8
e3dfd7fd-93f9-44d6-8fd2-de781d4ab63a
a5c16151-efaf-49dd-bbf4-bc545855f5cc
d68d5bc6-d3bd-4615-8fcf-d74286d1fe96
72120961-364a-4d25-9135-dfafa9f797a8
10630cc9-e4d8-47f4-8d4c-b4f043fbe18f
f842ff8b-775c-4e03-b792-107ed2e6f40d
1ee3f639-aa3b-4e8b-9611-49eac8cdb234
d13e5af6-d6e6-4dee-8014-937b27f04fa7
9e25473b-aaa1-4730-bcfc-85ac2cdecd95
957bbc36-a94b-4276-bad3-7411a0083c33
0d2c2ea7-9fea-4e94-a94f-f3e40e517c0a
89e55e0d-8eac-45c3-b074-2d53ec211ece
a112ded5-0931-4d6f-ab0d-8e67bdbaecae
fad6c37b-a8b5-418d-9ac7-2f9a45ac75ea
f32efa22-83b8-4fa9-a7b2-c935a8baf20a
4d52ffbb-3ae0-4803-937c-a9faec986b2f
ca004866-9103-4b7b-92a3-781095bef8b0
6e96fa60-5d6d-41c6-bc60-1d56ac046a0d
1b5e48e3-3531-4266-9591-0075d6d218ba
d5e8eb8e-f5df-4f77-a436-b29fc9826a47
8b5ff942-4c20-408c-bb07-8c008bed8492
fb4d7487-524f-4945-9c3c-b52a40258e3f
95b72be6-641c-4c28-8d85-a628fb1929c4
e59a485d-5b3b-493e-b4d9-f0bd4b9e00eb
d

In [23]:
#df_testIDs_url = pd.read_csv('data/url_testID.csv', sep='|', encoding='utf-8', keep_default_na=False)
#df_testIDs_ori_amp = pd.read_csv('data/ori_amp_url_testID.csv', sep='|', encoding='utf-8', keep_default_na=False)
#df_testIDs_amp_viewer = pd.read_csv('data/amp_viewer_url_testID-21092018.csv', sep='|', encoding='utf-8', keep_default_na=False)
#df_testIDs_amp_cdn = pd.read_csv('data/amp_cdn_url_testID_21092018.csv', sep='|', encoding='utf-8', keep_default_na=False)
#df_testIDs_amp_viewer_urlEUUS = pd.read_csv('data/amp_viewer_url_testID_EUUS-21072018.csv', sep='|', encoding='utf-8', keep_default_na=False)
df_testIDs_amp_cdn_urlEUUS = pd.read_csv('data/amp_cdn_url_testID_EUUS-22092018.csv', sep='|', encoding='utf-8', keep_default_na=False)


In [24]:
#df_results_ori_amp = getResultsFromTestIDs(df_testIDs_ori_amp)
#df_results_amp_viewer = getResultsFromTestIDs(df_testIDs_amp_viewer)
#df_results_amp_cdn = getResultsFromTestIDs(df_testIDs_amp_cdn)
#df_results_url = getResultsFromTestIDs(df_testIDs_url)
#df_results_url_EUUS = getResultsFromTestIDs(df_testIDs_urlEUUS)
#df_results_amp_viewer_url_EUUS = getResultsFromTestIDs(df_testIDs_amp_viewer_urlEUUS)
df_results_amp_cdn_url_EUUS = getResultsFromTestIDs(df_testIDs_amp_cdn_urlEUUS)


In [25]:
#df_results_url.to_csv("data/url_results" + time.strftime("%d%m%Y") + ".csv", sep='|', encoding='utf-8', index=False)
#df_results_ori_amp.to_csv("data/ori_amp_url_results.csv", sep='|', encoding='utf-8', index=False)
#df_results_amp_viewer.to_csv("data/amp_viewer_url_results-2109201.csv", sep=';', encoding='utf-8', index=False)
#df_results_amp_cdn.to_csv("data/amp_cdn_url_results21092018.csv", sep='|', encoding='utf-8', index=False)
#df_results_url_EUUS.to_csv("data/url_results_EUUS" + time.strftime("%d%m%Y") + ".csv", sep='|', encoding='utf-8', index=False)
#df_results_amp_viewer_url_EUUS.to_csv("data/amp_viewer_url_results-" + time.strftime("%d%m%Y") + ".csv", sep='|', encoding='utf-8', index=False)
df_results_amp_cdn_url_EUUS.to_csv("data/amp_cdn_url_results-" + time.strftime("%d%m%Y") + ".csv", sep=';', encoding='utf-8', index=False)
